In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace
from pyspark import StorageLevel
import findspark
import os
import re
import shutil

findspark.init()
findspark.find()

'/home/saulo/anaconda3/envs/pyspark_env/lib/python3.7/site-packages/pyspark'

In [2]:
# Create SparkSession
spark = SparkSession.builder\
        .appName("Aula_2")\
        .getOrCreate()
		
spark.sparkContext.setLogLevel("WARN")

# Desabilita _SUCCESS
# spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "true") 


2022-06-14 21:11:54 WARN  Utils:66 - Your hostname, PC resolves to a loopback address: 127.0.1.1; using 172.25.176.6 instead (on interface eth0)
2022-06-14 21:11:54 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2022-06-14 21:11:55 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


##### SCIMAGO e JCS
Limpeza e padronização das colunas dos dois arquivos

Alteração de tipagem dos dados que são númericos e foram lidos como string (incluindo alterar virgulas por pontos)

Junção dos dois arquivos

Salvamento do arquivo final

Deve ser usada a API DataFrame do Spark.


In [3]:
dir_input = os.path.join(os.getcwd(),'input')
dir_output = os.path.join(os.getcwd(),'output')
dir_config = os.path.join(os.getcwd(),'config')

file_scimago = 'scimagojr 2020.csv'
file_jcs = 'jcs_2020.csv'

df_raw_scimago = spark.read.csv(os.path.join(dir_input,file_scimago), sep=';', encoding='utf-8', header=True)
df_raw_jcs = spark.read.csv(os.path.join(dir_input,file_jcs), sep=';', encoding='utf-8', header=True)


In [4]:
# Renomear colunas
df_scimago = df_raw_scimago.toDF(*(column\
	.lower()
	.replace(' ', '_')
	.replace('.','')
	.replace('/','')
	.replace('__','_')
	for column in df_raw_scimago.columns))

df_jcs = df_raw_jcs.toDF(*(column\
	.lower()
	.replace(' ', '_')
	.replace('.','')
	.replace('/','')
	.replace('__','_')
	for column in df_raw_jcs.columns))


In [5]:
# Cria uma tabela temporaria para usar sql
df_scimago.createOrReplaceTempView("df_scimago")
df_jcs.createOrReplaceTempView("df_jcs")

df_scimago_final = spark.sql("""
SELECT DISTINCT
 title
,type
,cast(REGEXP_REPLACE(sjr, ',', '.') AS DECIMAL(18,2)) AS scimago_value
FROM df_scimago
WHERE sjr IS NOT NULL
""").cache()


df_jcs_final = spark.sql("""
SELECT DISTINCT
 full_journal_title
,CAST(REGEXP_REPLACE(journal_impact_factor, ',', '.') AS DECIMAL(18,2)) AS jcs_value
FROM df_jcs
""").cache()


In [6]:
df_scimago_final.createOrReplaceTempView("df_scimago_final")
df_jcs_final.createOrReplaceTempView("df_jcs_final")

df_scimago_jcs = spark.sql("""
SELECT DISTINCT
T1.title
,T1.type
,T1.scimago_value
,T2.jcs_value
FROM df_scimago_final T1
INNER JOIN df_jcs_final T2
ON lower(T1.title) = lower(t2.full_journal_title)
""").cache()

print(df_scimago_jcs.count())
df_scimago_jcs.show(10)



9858
+--------------------+-------+-------------+---------+
|               title|   type|scimago_value|jcs_value|
+--------------------+-------+-------------+---------+
|     Nature Genetics|journal|        18.86|    38.33|
|Progress in Retin...|journal|         7.20|    21.20|
|Current Opinion i...|journal|         5.13|     8.38|
|         SIAM Review|journal|         4.68|    10.78|
|npj Quantum Infor...|journal|         3.40|     7.39|
|  Nanoscale Horizons|journal|         2.99|    10.99|
|     Brain Pathology|journal|         1.99|     6.51|
|Radiotherapy and ...|journal|         1.89|     6.28|
|Plastic and Recon...|journal|         1.84|     4.73|
|International Jou...|journal|         1.71|     5.58|
+--------------------+-------+-------------+---------+
only showing top 10 rows



In [7]:
print(df_scimago_jcs.rdd.getNumPartitions())

#Export to CSV
df_scimago_jcs.repartition(1).write.csv(os.path.join(dir_output,'scimago_jcs'),
	header=True,
	sep=';',
	mode='Overwrite')

print(os.listdir(os.path.join(dir_output,'scimago_jcs')))

200
['.part-00000-51ceadae-3230-4e0a-b82d-c915e594bb81-c000.csv.crc', '._SUCCESS.crc', 'part-00000-51ceadae-3230-4e0a-b82d-c915e594bb81-c000.csv', '_SUCCESS']


In [8]:
# Renomea arquivo
for root, dirs, files in os.walk(os.path.join(dir_output, 'scimago_jcs')):
	for file_name in files:	
		if re.search('\.csv$', file_name):
			os.rename(os.path.join(dir_output,'scimago_jcs', file_name,), os.path.join(dir_output, 'scimago_jcs.csv'))

shutil.rmtree(os.path.join(dir_output, 'scimago_jcs')) # Remove o diretorio e todos os arquivos nele
os.listdir(os.path.join(dir_output))
			

# df_scimago_jcs.toPandas().to_csv(os.path.join(dir_output,'scimago_jcs_pandas.csv'),
# 	header=True,
# 	sep=';',
# 	encoding='utf-8',
# 	index=False)			

['scimago_jcs.csv', 'scimago_jcs_pandas.csv']

In [9]:
df_scimago_final.unpersist()
df_jcs_final.unpersist()
df_scimago_jcs.unpersist()

DataFrame[title: string, type: string, scimago_value: decimal(18,2), jcs_value: decimal(18,2)]